In [1]:
# CLAP Score compare
import numpy as np
import librosa
import torch
import CLAP.src.laion_clap as laion_clap

# quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)


def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)

model = laion_clap.CLAP_Module(enable_fusion=False)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model.load_ckpt(model_id=1)

Load our best checkpoint in the paper.
Download completed!
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.weight 	 Loaded
aud

In [ ]:
class Config():
    def __init__():
        self.local_loss
        self.gather_with_grad
        self.rank
        self.world_size
        self.horovod
        self.clap_mlploss
        self.kappa

device = torch.device('cuda')

loss = laion_clap.CLIPLoss(
    local_loss=args.local_loss, # default False
    gather_with_grad=args.gather_with_grad, # default False
    cache_labels=True,
    rank=args.rank, # default 0
    world_size=args.world_size, # default 1
    use_horovod=args.horovod, # default False
    mlp_loss=args.clap_mlploss, # default False
    weight_loss_kappa=args.kappa, # default 0
)

In [ ]:


model, model_cfg = create_model(
    args.amodel, # audio module : RN50
    args.tmodel, # text module  : transformer
    args.pretrained, # ""가 디폴트긴 한데..
    # precision=args.precision, # fp32 
    device=device,
    # jit=args.torchscript, # False
    # force_quick_gelu=args.force_quick_gelu, # False
    openai_model_cache_dir=os.path.expanduser(args.openai_model_cache_dir),
    # skip_params=True,
    # pretrained_audio=args.pretrained_audio, # ""
    # pretrained_text=args.pretrained_text,   # ""
    # enable_fusion=args.enable_fusion,       # False
    # fusion_type=args.fusion_type            # 'None'
)


exclude = (
    lambda n, p: p.ndim < 2
    or "bn" in n
    or "ln" in n
    or "bias" in n
    or "logit_scale" in n
)
include = lambda n, p: not exclude(n, p)

named_parameters = list(model.named_parameters())


gain_or_bias_params = [
    p for n, p in named_parameters if exclude(n, p) and p.requires_grad
]
rest_params = [p for n, p in named_parameters if include(n, p) and p.requires_grad]

optimizer = get_optimizer(
    [
        {"params": gain_or_bias_params, "weight_decay": 0.0},
        {"params": rest_params, "weight_decay": args.wd},
    ],
    lr=args.lr,
    betas=(args.beta1, args.beta2),
    eps=args.eps,
    momentum=args.momentum,
    optimizer_name=args.optimizer,
)

scheduler = cosine_lr(optimizer, args.lr, args.warmup, total_steps)




In [ ]:
for idx, batch in enumerate(dataloader):
    audios = batch
    texts = batch['text']

    with autocast():
        (
            audio_features,
            text_features,
            audio_features_mlp,
            text_features_mlp,
            logit_scale_a,
            logit_scale_t,
        ) = model(audios, texts, device)
    